# Review: Querying an API endpoint

### Mapbox Geocoding API

Services like Google Maps and Mapbox have various APIs that let you access its services through code instead of through GUI apps. This one from Mapbox lets you look up the latitude-longitude coordinates of street addresses.

It works similarly to the earthquakes example, but with query parameters added to the URL endpoint!

**API documentation:**  
https://www.mapbox.com/api-documentation/#geocoding

**API endpoint:**  
https://api.mapbox.com/geocoding/v5/mapbox.places

**API endpoint with query parameters:**  
https://api.mapbox.com/geocoding/v5/mapbox.places/Wurster+Hall.json?access_token=pk.eyJ1IjoiY3AyNTVkZW1vIiwiYSI6ImRPcTlnTUEifQ.3C0d0Nk_rcwV-8JF29PU-w


## Setting up API Keys for this session

You can get your own access key by signing up for a Mapbox account. Here is a link for that (Go ahead and do it now):

https://www.mapbox.com/signin/?route-to=%22/account/access-tokens%22

The Census Bureau has many APIs that can be used directly from Python - we will explore one later in class today.  You'll need to request an API key here, if you don't already have this one: 

https://api.census.gov/data/key_signup.html

Go ahead and do this, and check your email when we come to that part of the session that needs this.

## Reviewing the Mapbox API endpoint for geocoding

In [1]:
import json      # library for working with JSON-formatted text strings
import requests  # library for accessing content from web URLs

import pprint    # library for cleanly printing Python data structures
pp = pprint.PrettyPrinter()

In [2]:
# we have to encode the search query so that it can be passed as a URL, 
# with spaces and other special characters removed

endpoint = 'https://api.mapbox.com/geocoding/v5/mapbox.places/'

address = 'Wurster Hall'

params = {'limit': 1,
          'access_token': 'pk.eyJ1IjoiY3AyNTVkZW1vIiwiYSI6ImRPcTlnTUEifQ.3C0d0Nk_rcwV-8JF29PU-w'}

url = requests.Request('GET', endpoint+address+'.json', params=params).prepare().url
print(url)

https://api.mapbox.com/geocoding/v5/mapbox.places/Wurster%20Hall.json?limit=1&access_token=pk.eyJ1IjoiY3AyNTVkZW1vIiwiYSI6ImRPcTlnTUEifQ.3C0d0Nk_rcwV-8JF29PU-w


In [3]:
# download and parse the results

response = requests.get(url)
results = response.text
data = json.loads(results)

print(data)

{'type': 'FeatureCollection', 'query': ['wurster', 'hall'], 'features': [{'id': 'poi.1359351', 'type': 'Feature', 'place_type': ['poi'], 'relevance': 1, 'properties': {'tel': '(510) 642-0831', 'address': '230 Wurster Hall #1820', 'category': 'college, university', 'landmark': True, 'maki': 'college'}, 'text': 'Wurster Hall / College of Environmental Design', 'place_name': 'Wurster Hall / College of Environmental Design, 230 Wurster Hall #1820, Berkeley, California 94720, United States', 'center': [-122.25488, 37.87082], 'geometry': {'type': 'Point', 'coordinates': [-122.25488, 37.87082]}, 'context': [{'id': 'postcode.1038777018848930', 'text': '94720'}, {'id': 'place.4062647275990170', 'wikidata': 'Q484678', 'text': 'Berkeley'}, {'id': 'region.3591', 'short_code': 'US-CA', 'wikidata': 'Q99', 'text': 'California'}, {'id': 'country.3145', 'short_code': 'us', 'wikidata': 'Q30', 'text': 'United States'}]}], 'attribution': 'NOTICE: © 2018 Mapbox and its suppliers. All rights reserved. Use o

In [4]:
# print it more nicely

pp.pprint(data)

{'attribution': 'NOTICE: © 2018 Mapbox and its suppliers. All rights reserved. '
                'Use of this data is subject to the Mapbox Terms of Service '
                '(https://www.mapbox.com/about/maps/). This response and the '
                'information it contains may not be retained. POI(s) provided '
                'by Foursquare.',
 'features': [{'center': [-122.25488, 37.87082],
               'context': [{'id': 'postcode.1038777018848930', 'text': '94720'},
                           {'id': 'place.4062647275990170',
                            'text': 'Berkeley',
                            'wikidata': 'Q484678'},
                           {'id': 'region.3591',
                            'short_code': 'US-CA',
                            'text': 'California',
                            'wikidata': 'Q99'},
                           {'id': 'country.3145',
                            'short_code': 'us',
                            'text': 'United States',
         

In [5]:
# pull out the lat-lon coordinates

for r in data['features']:
    coords = r['geometry']['coordinates']
    print(coords)

[-122.25488, 37.87082]


## Using the Mapbox Python SDK for Geocoding

So far the discussion of APIs has been based on just accessing the API endpoints, which could be done with anything that can access a properly specified url.  Even just your browser.

Now we will look at APIs from a different perspective, one that is more Pythonic.  We will first need to install the Mapbox SDK.

In [7]:
!pip install mapbox

    100% |████████████████████████████████| 133kB 7.8MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 7.8MB/s ta 0:00:011
    100% |████████████████████████████████| 4.7MB 9.7MB/s eta 0:00:01
  Running setup.py bdist_wheel for iso3166 ... done
  Stored in directory: /Users/paul/Library/Caches/pip/wheels/05/75/28/62c7e94498ff866a39dbaec5db14fc5b3214a6506e69254a63
  Running setup.py bdist_wheel for cachecontrol ... done
  Stored in directory: /Users/paul/Library/Caches/pip/wheels/36/bd/5f/dbbee4f2d51f97ecd12a363f870361179cb1fd4bc1174ea08a
Successfully built iso3166 cachecontrol
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


You will need to manage access tokens for Mapbox APIs (and for many others).  Read this for more information:

https://github.com/mapbox/mapbox-sdk-py/blob/master/docs/access_tokens.md

### Forward Geocoding

Forward geocoding is the one we have looked at so far using an ALI endpoint.  Let's look at it again using the Mapbox Python SDK.

In [8]:
from mapbox import Geocoder
import os

First - a brief digression -- these access tokens should not be publicly exposed, like on a web page or github, since others could get them and abuse them, and they are connected to you, so you might get blocked (blacklisted) by the API providers.  So -- the best way to handle this is to store the API Key or access token in an environment variable.  Since we are using datahub and each account is accessible only via Calnet Authentication, this is not such a big deal, but you should be aware of it and manage it differently on your own computer - by storing them in environment variables on your computer.  

Each operating system handled environment variables differently, and that is something you should learn on your own.  Below is a simple example of setting them temporarily.  Usually you would add them to a file that gets read when you start a command shell.  For example on my mac it is in .bash_profile.

In [9]:
os.environ['MAPBOX_ACCESS_TOKEN'] = "pk.eyJ1IjoiY3AyNTVkZW1vIiwiYSI6ImRPcTlnTUEifQ.3C0d0Nk_rcwV-8JF29PU-w"

In [11]:
os.environ['MAPBOX_ACCESS_TOKEN']

'pk.eyJ1IjoiY3AyNTVkZW1vIiwiYSI6ImRPcTlnTUEifQ.3C0d0Nk_rcwV-8JF29PU-w'

OK, now we can get to work on the geocoding... let's try geocoding Freehouse Restaurant, which is at 2700 Bancroft Way.

In [12]:
geocoder = Geocoder(access_token = os.environ['MAPBOX_ACCESS_TOKEN'])

In [14]:
response = geocoder.forward('2700 Bancroft Way, Berkeley, CA 94704', limit=1)

In [15]:
results = response.text

In [16]:
data = json.loads(results)
pp.pprint(data)

{'attribution': 'NOTICE: © 2018 Mapbox and its suppliers. All rights reserved. '
                'Use of this data is subject to the Mapbox Terms of Service '
                '(https://www.mapbox.com/about/maps/). This response and the '
                'information it contains may not be retained. POI(s) provided '
                'by Foursquare.',
 'features': [{'address': '2700',
               'center': [-122.25425, 37.868969],
               'context': [{'id': 'neighborhood.285002',
                            'text': 'Telegraph Avenue'},
                           {'id': 'postcode.15012008988668840',
                            'text': '94704'},
                           {'id': 'place.4062647275990170',
                            'text': 'Berkeley',
                            'wikidata': 'Q484678'},
                           {'id': 'region.3591',
                            'short_code': 'US-CA',
                            'text': 'California',
                            'w

Another random address to geocode:

In [18]:
response = geocoder.forward('120 East 13th Street, Manhattan, New York, New York 10003', limit = 1)
results = response.text
first = response.geojson()['features'][0]
print(first['place_name'])
print([coord for coord in first['geometry']['coordinates']])

120 East 13th Street, Manhattan, New York, New York 10003, United States
[-73.988893, 40.733003]


### Your turn

Try geocoding 3 addresses you know. 

Can you think of a good way to write a loop to geocode those addresses and add them to a dataframe?  Give it a try!

In [19]:
import pandas as pd
data=pd.DataFrame(index=[],columns=['address','lat','lon'])
data

,address,lat,lon


In [24]:
address=['3028 Regent St., Berkeley, California, 94705', '1511 Julia St, Berkeley, California, 94703','2700 Bancroft, Berkeley, California, 94704',]

In [25]:
for i in range(len(address)):
    geoadress=geocoder.forward(address[i], limit = 1).geojson()['features'][0]['geometry']['coordinates']
    data.loc[i,'address']=address[i]
    data.loc[i,'lat']=geoadress[0]
    data.loc[i,'lon']=geoadress[1]
data

,address,lat,lon
0,"3028 Regent St., Berkeley, California, 94705",-122.257,37.8545
1,"1511 Julia St, Berkeley, California, 94703",-122.279,37.8545
2,"2700 Bancroft, Berkeley, California, 94704",-122.254,37.869


### Reverse Geocoding

Reverse geocoding does what it sounds like. It takes coordinates and returns an address or other place type.  Options for place type include: country, region, postcode, district, place, locality, neighborhood, address,and poi.

Let's start by using the coordinates of the address we just got from forward geocoding, and get the address back from reverse geocoding of those coordinates.  

**Notice that the response argument is not JSON, it is GeoJson.  It is an expanded version of JSON that contains geometry.  It is the web standard for geospatial data.  Think: open data version of ESRI Shapefile, but optimized for the web.**  See http://geojson.org/ for more info.  We will deal more with GeoJson formatted files as we get into more web mapping tools soon.

In [26]:
response = geocoder.reverse(lon=-73.988893, lat=40.733003)
features = response.geojson()['features']
features

[{'id': 'address.4216698541380788',
  'type': 'Feature',
  'place_type': ['address'],
  'relevance': 1,
  'properties': {},
  'text': 'East 13th Street',
  'place_name': '120 East 13th Street, Manhattan, New York, New York 10003, United States',
  'center': [-73.988893, 40.733003],
  'geometry': {'type': 'Point', 'coordinates': [-73.988893, 40.733003]},
  'address': '120',
  'context': [{'id': 'neighborhood.2103290', 'text': 'Greenwich Village'},
   {'id': 'locality.12696928000137850',
    'wikidata': 'Q11299',
    'text': 'Manhattan'},
   {'id': 'postcode.13482670360296810', 'text': '10003'},
   {'id': 'place.15278078705964500', 'wikidata': 'Q60', 'text': 'New York'},
   {'id': 'region.3866',
    'short_code': 'US-NY',
    'wikidata': 'Q1384',
    'text': 'New York'},
   {'id': 'country.3145',
    'wikidata': 'Q30',
    'short_code': 'us',
    'text': 'United States'}]},
 {'id': 'neighborhood.2103290',
  'type': 'Feature',
  'place_type': ['neighborhood'],
  'relevance': 1,
  'propert

In [27]:
for f in features:
    print('{place_name}'.format(**f))

120 East 13th Street, Manhattan, New York, New York 10003, United States
Greenwich Village, Manhattan, New York, New York 10003, United States
Manhattan, 10003, New York, New York, United States
New York, New York 10003, United States
New York, New York, United States
New York, United States
United States


Now let's look up Freehouse Restaurant's address from its Lat Long

In [28]:
response = geocoder.reverse(lon=-122.25425, lat=37.868969, types=['address'])  
features = response.geojson()['features']
features[0]['place_name']

'2700 Bancroft Way, Berkeley, California 94704, United States'

### Your turn

Try forward geocoding an address, and then reverse geocoding the coordinates to get the address back. Use your home address and see if it works.  Try a second address.

But first - check in here: http://bitly.com/cp255

### Getting Directions from Mapbox

You can put in start and end locations and call the Mapbox directions API to get suggested directions as a sequence of coordinates you could plot on a map.

In [29]:
import mapbox
help(mapbox.Directions)

Help on class Directions in module mapbox.services.directions:

class Directions(mapbox.services.base.Service)
 |  Access to the Directions v5 API.
 |  
 |  Method resolution order:
 |      Directions
 |      mapbox.services.base.Service
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  directions(self, features, profile='mapbox/driving', alternatives=None, geometries=None, overview=None, steps=None, continue_straight=None, waypoint_snapping=None, annotations=None, language=None, **kwargs)
 |      Request directions for waypoints encoded as GeoJSON features.
 |      
 |      Parameters
 |      ----------
 |      features : iterable
 |          An collection of GeoJSON features
 |      profile : str
 |          Name of a Mapbox profile such as 'mapbox.driving'
 |      alternatives : bool
 |          Whether to try to return alternative routes, default: False
 |      geometries : string
 |          Type of geometry returned (geojson, polyline, polyline6)
 |      overview : 

In [30]:
from mapbox import Directions

In [31]:
service = Directions()

In [32]:
origin = {
        'type': 'Feature',
        'properties': {'name': 'Portland, OR'},
        'geometry': {
        'type': 'Point',
        'coordinates': [-122.7282, 45.5801]}}
destination = {
    'type': 'Feature',
    'properties': {'name': 'Bend, OR'},
    'geometry': {
    'type': 'Point',
    'coordinates': [-121.3153, 44.0582]}}

In [33]:
response = service.directions([origin, destination],'mapbox/driving')

In [34]:
driving_routes = response.geojson()

In [35]:
driving_routes

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'LineString',
    'coordinates': [(45.57994, -122.72832),
     (45.56995, -122.69555),
     (45.52473, -122.66403),
     (45.53161, -122.56801),
     (45.54728, -122.55336),
     (45.53943, -122.41837),
     (45.45508, -122.37623),
     (45.3759, -122.2218),
     (45.36648, -122.15453),
     (45.37977, -122.04793),
     (45.30527, -121.87119),
     (45.3123, -121.79323),
     (45.30066, -121.73499),
     (45.15902, -121.66228),
     (45.10711, -121.55876),
     (45.05603, -121.51364),
     (45.02217, -121.51322),
     (44.86717, -121.42421),
     (44.78583, -121.32094),
     (44.76025, -121.22736),
     (44.71869, -121.22773),
     (44.72038, -121.17616),
     (44.65345, -121.12858),
     (44.45997, -121.19954),
     (44.27523, -121.16913),
     (44.05817, -121.31533)]},
   'properties': {'distance': 269397.3, 'duration': 11651.9}}]}

**Soon we will begin working with GeoJson files and mapping them... but not today.

## Reverse geocoding to FIPS

We'll use the FCC's Census Block Conversions API to turn lat/long into a block FIPS code. FIPS codes contain from left to right: the location's 2-digit state code, 3-digit county code, 6-digit census tract code, and 4-digit census block code (the first digit of which is the census block group code). Now you can join your data to tract (etc) level census data without doing a spatial join.

- Documentation: https://geo.fcc.gov/api/census/

In [36]:
url = 'https://geo.fcc.gov/api/census/block/find?latitude=34.537094&longitude=-82.630303&format=json'
response = requests.get(url)
data = response.json()
data

{'messages': ["FCC0001: The coordinate lies on the boundary of mulitple blocks, the block contains the clicked location is selected. For a complete list use showall=true to display 'intersection' element in the Block"],
 'Block': {'FIPS': '450070003002024',
  'bbox': [-82.636071, 34.535797, -82.630107, 34.540163]},
 'County': {'FIPS': '45007', 'name': 'Anderson'},
 'State': {'FIPS': '45', 'code': 'SC', 'name': 'South Carolina'},
 'status': 'OK',
 'executionTime': '0'}

In [29]:
data['Block']['FIPS']

'450070003002024'

In [30]:
data['County']['name']

'Anderson'

### Your turn: 

1. Look up the latitude and longitude of your home address using forward geocoding
2. Use the latitude and longitude from that to get the census block of your home address
3. Challenge problem: create a dataframe with 3 street addresses and a description column with labels for these locations, and write code that will iterate through them, get the latitude and longitude for each, and the FIPS block code, County name, and write these addresses, lat and long, and FIPS Block code and County names to a dataframe.

### Reminder about reading for next session

Some of you have considerable experience with GIS already, and are very comfortable with geographic coordinate system and map projection, and managing these in spatial data.  If so, you might not need to spend much time with the readings for next session, but if not, PLEASE read these and get comfortable with them before class or you will get lost quickly.

https://en.wikipedia.org/wiki/Geographic_coordinate_system

https://en.wikipedia.org/wiki/Map_projection

We will also be using the GeoPandas Python library, so browse this.  We will cover it in detail over the next two sessions. http://geopandas.org/